In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
)
from webdriver_manager.chrome import ChromeDriverManager
import time

import pandas as pd
import numpy as np
import time
import json

In [2]:
def handle_cookie_consent():
    try:
        # Wait for the cookie consent banner to be present
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "onetrust-banner-sdk"))
        )
        # Click the "Accept" button
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@id='onetrust-accept-btn-handler']"))
        )
        accept_button.click()
        print("Cookie consent accepted.")
        # Wait for the banner to disappear
        WebDriverWait(driver, 5).until(
            EC.invisibility_of_element_located((By.ID, "onetrust-banner-sdk"))
        )
    except (TimeoutException, NoSuchElementException):
        # If the banner does not appear, proceed
        print("No cookie consent banner to accept.")

In [3]:
json_file_path = 'beer_reviews.json'
with open(json_file_path, 'w') as f:
    f.write('[')

In [4]:
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options = chrome_options)

# Navigate to the Beer Advocate Top Rated page
driver.get("https://www.ratebeer.com/top-beers")

WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located(
            (By.XPATH, "//div[contains(@class,'DataTable__Row')]/div[2]/a")
        )
    )

beers = driver.find_elements(By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "MuiTypography-body2", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "text-500", " " ))]')

# Loop through all beers
for i in range(len(beers)):
    refresh_beers = driver.find_elements(By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "MuiTypography-body2", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "text-500", " " ))]')
    product_name = refresh_beers[i].text

    # Within beer page
    refresh_beers[i].click()

    handle_cookie_consent()

    dropdown = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "MuiSelect-selectMenu"))
        )
    
    #dropdown = driver.find_element(By.CLASS_NAME, "MuiSelect-selectMenu")
    dropdown.click()

    # Wait for the dropdown options to be visible
    option_100 = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, "//li[contains(text(), '100')]"))
    )
    option_100.click()

    num_items = driver.find_element(By.CSS_SELECTOR, ".MuiTablePagination-selectRoot + .MuiTypography-colorInherit").text
    num_next = int(np.ceil(int(num_items.split(" ")[-1]) / 100))

    for j in range(num_next):
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".fb-0 .MuiButton-label"))
        )

        show_more_buttons = driver.find_elements(By.CSS_SELECTOR, ".fb-0 .MuiButton-label")
        show_more_buttons = [button for button in show_more_buttons if button.text == "Show more"]
                
        for button in show_more_buttons:
            button.click()

        # Get all comments and store as a list
        comments_elements = driver.find_elements(By.CSS_SELECTOR, ".pre-wrap.MuiTypography-body1")
        product_reviews = [comment.text for comment in comments_elements]

        rating_elements = driver.find_elements(By.CSS_SELECTOR, ".bRPQdN.MuiTypography-subtitle1")
        ratings = [rating.text for rating in rating_elements]

        beer_data_list = []
        
        for review, rating in zip(product_reviews, ratings):
            beer_data = {
                'product_name': product_name,
                'product_review': review,
                'rating': rating
            }
            beer_data_list.append(beer_data)

        with open(json_file_path, 'a') as f:
            for beer_data in beer_data_list:
                f.write(json.dumps(beer_data) + ',\n')

        if j + 1 < num_next:

            button_to_click = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".MuiIconButton-colorInherit + .MuiIconButton-colorInherit"))
            )
            button_to_click.click()

    driver.back()
    
with open(json_file_path, 'rb+') as f:
        f.seek(-2, 2)  # Go to the second last byte
        f.write(b'\n]')   

driver.quit()

Cookie consent accepted.
No cookie consent banner to accept.
No cookie consent banner to accept.
No cookie consent banner to accept.
No cookie consent banner to accept.


ElementClickInterceptedException: Message: element click intercepted: Element <span class="MuiButton-label">...</span> is not clickable at point (467, 12). Other element would receive the click: <div class="RateBeerBestBanner___StyledDiv-jAauHc gBedvV fa-c fj-sb f-wrap py-4 px-4">...</div>
  (Session info: chrome=129.0.6668.70)
Stacktrace:
0   chromedriver                        0x0000000101094274 cxxbridge1$str$ptr + 1907280
1   chromedriver                        0x000000010108c75c cxxbridge1$str$ptr + 1875768
2   chromedriver                        0x0000000100ca0260 cxxbridge1$string$len + 89488
3   chromedriver                        0x0000000100ce9cd4 cxxbridge1$string$len + 391172
4   chromedriver                        0x0000000100ce8328 cxxbridge1$string$len + 384600
5   chromedriver                        0x0000000100ce653c cxxbridge1$string$len + 376940
6   chromedriver                        0x0000000100ce5b58 cxxbridge1$string$len + 374408
7   chromedriver                 

In [2]:
with open('beer_reviews.json', 'r') as file:
    beer_reviews = json.load(file)

In [3]:
pd.DataFrame(beer_reviews)

,product_name,product_review,rating
0,Toppling Goliath Kentucky Brunch,"You need personal informations from companies,...",3.3
1,Toppling Goliath Kentucky Brunch,"Bottle after MBCC 2024. Black colour, malty ar...",4.5
2,Toppling Goliath Kentucky Brunch,Thank you for sharing this Chris - Black with ...,4.3
3,Toppling Goliath Kentucky Brunch,"Boxed beer at home, proper glassware. Pitch bl...",4.7
4,Toppling Goliath Kentucky Brunch,"From backlog. (As 2018 Vintage) 0,3 litre Bott...",4.9
...,...,...,...
5187,Westvleteren 12,.................................................,4.0
5188,Westvleteren 12,Usually I prefer the smaller beers from within...,4.8
5189,Westvleteren 12,"What can I say, the absolute best Trappist pro...",5.0
5190,Westvleteren 12,"Chestnut brown. Aroma shows alcohol, wood, hi...",4.4


In [9]:
driver.quit()

In [9]:
beer_reviews_df = pd.DataFrame(beer_reviews)
beer_reviews_df.value_counts('product_name')

product_name
Westvleteren 12                     4280
Närke Kaggen Stormaktsporter         628
Toppling Goliath Kentucky Brunch     181
Schramm's The Heart of Darkness      103
Name: count, dtype: int64

In [13]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
    WebDriverException,
)
from webdriver_manager.chrome import ChromeDriverManager
import time
import numpy as np
import json

def handle_cookie_consent():
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "onetrust-banner-sdk"))
        )
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@id='onetrust-accept-btn-handler']"))
        )
        accept_button.click()
        WebDriverWait(driver, 5).until(
            EC.invisibility_of_element_located((By.ID, "onetrust-banner-sdk"))
        )
    except (TimeoutException, NoSuchElementException, WebDriverException):
        pass

def handle_ratebeer_banner():
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'RateBeerBestBanner___StyledDiv-jAauHc'))
        )
        close_button = driver.find_element(By.CSS_SELECTOR, '.RateBeerBestBanner___StyledDiv-jAauHc .MuiIconButton-root')
        close_button.click()
        WebDriverWait(driver, 5).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, 'RateBeerBestBanner___StyledDiv-jAauHc'))
        )
    except (TimeoutException, NoSuchElementException, WebDriverException):
        pass

json_file_path = 'beer_reviews.json'
with open(json_file_path, 'w', encoding='utf-8') as f:
    f.write('[')

chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), options=chrome_options
)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# Navigate to the RateBeer Top Beers page
driver.get("https://www.ratebeer.com/top-beers")

handle_cookie_consent()

WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located(
        (By.XPATH, "//div[contains(@class,'DataTable__Row')]/div[2]/a")
    )
)

# Get the list of beers and their links
beer_elements = driver.find_elements(By.XPATH, "//div[contains(@class,'DataTable__Row')]/div[2]/a")
beers = []
for beer_element in beer_elements:
    full_text = beer_element.text
    product_name = full_text.split('\n')[0]  # Take the first line before the newline
    product_link = beer_element.get_attribute('href')
    beers.append({'name': product_name, 'link': product_link})

total_reviews_collected = 0
max_reviews = 5000  # Set your desired number of reviews to collect
first_review = True  # Flag to handle commas in JSON

# Loop through all beers
for i in range(len(beers)):
    product_name = beers[i]['name']
    product_link = beers[i]['link']

    # Navigate to the beer page
    driver.get(product_link)

    handle_cookie_consent()
    handle_ratebeer_banner()

    # Wait for the dropdown to be present
    try:
        dropdown = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "MuiSelect-selectMenu"))
        )
        dropdown.click()

        # Wait for the dropdown options to be visible
        option_100 = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//li[contains(text(), '100')]"))
        )
        option_100.click()
    except (TimeoutException, NoSuchElementException, WebDriverException):
        continue

    # Get the total number of reviews
    try:
        num_items_text = driver.find_element(By.CSS_SELECTOR, ".MuiTablePagination-selectRoot + .MuiTypography-colorInherit").text
        total_reviews = int(num_items_text.split(" ")[-1])
        num_pages = int(np.ceil(total_reviews / 100))
    except (NoSuchElementException, ValueError, WebDriverException):
        continue

    for j in range(num_pages):
        time.sleep(2)
        # Expand all "Show more" buttons
        while True:
            try:
                show_more_button = driver.find_element(By.XPATH, "//span[text()='Show more']")
                driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button)
                show_more_button.click()
                time.sleep(1)
            except NoSuchElementException:
                break
            except ElementClickInterceptedException:
                handle_ratebeer_banner()
                driver.execute_script("arguments[0].click();", show_more_button)
            except Exception as e:
                print(f"Could not click 'Show more' button: {e}")
                break

        # Get all comments and store as a list
        comments_elements = driver.find_elements(By.CSS_SELECTOR, ".pre-wrap.MuiTypography-body1")
        product_reviews = [comment.text for comment in comments_elements]

        rating_elements = driver.find_elements(By.CSS_SELECTOR, ".bRPQdN.MuiTypography-subtitle1")
        ratings = [rating.text for rating in rating_elements]

        beer_data_list = []

        for review, rating in zip(product_reviews, ratings):
            beer_data = {
                'product_name': product_name,
                'product_review': review,
                'rating': rating
            }
            beer_data_list.append(beer_data)
            total_reviews_collected += 1

            if total_reviews_collected >= max_reviews:
                break

        # Write data to the JSON file
        with open(json_file_path, 'a', encoding='utf-8') as f:
            for beer_data in beer_data_list:
                if not first_review:
                    f.write(',\n')
                json.dump(beer_data, f, ensure_ascii=False)
                first_review = False
        beer_data_list = []

        if total_reviews_collected >= max_reviews:
            break

        if j + 1 < num_pages:
            try:
                button_to_click = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".MuiIconButton-colorInherit + .MuiIconButton-colorInherit"))
                )
                handle_ratebeer_banner()
                button_to_click.click()
                time.sleep(1)
            except ElementClickInterceptedException:
                handle_ratebeer_banner()
                driver.execute_script("arguments[0].click();", button_to_click)
            except Exception as e:
                print(f"Could not click 'Next' button: {e}")
                break

    if total_reviews_collected >= max_reviews:
        break

# Close the JSON array properly
with open(json_file_path, 'a', encoding='utf-8') as f:
    f.write(']')

driver.quit()

In [14]:
# Load the JSON data into a DataFrame
with open('beer_reviews.json', 'r', encoding='utf-8') as file:
    beer_reviews = json.load(file)

df = pd.DataFrame(beer_reviews)
print(df['product_name'].value_counts())

product_name
Westvleteren 12                     4088
Närke Kaggen Stormaktsporter         628
Toppling Goliath Kentucky Brunch     181
Schramm's The Heart of Darkness      103
Name: count, dtype: int64
